Loading the necessary packages

In [5]:
from __future__ import print_function

import pandas as pd
import numpy as np

from src import tracker, rna, terra, loading

from genepy import terra
from genepy.utils import helper as h
from genepy.google import gcp
from genepy import rna

from genepy.google.google_sheet import dfToSheet
from gsheets import Sheets

from taigapy import TaigaClient
import dalmatian as dm

from bokeh.plotting import output_notebook

from biomart import BiomartServer
import io

from IPython.display import Image,display

%load_ext autoreload
%autoreload 2

tc = TaigaClient()
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

immutable parameters (user specific)

In [87]:
samplesetname="21Q2"
# current age at which to consider the sample already loaded in previous release
maxage='2020-11-01'

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)


# GP storage buckets
#workspace2="broad-firecloud-ccle/CCLE_DepMap_RNAseq"
#workspace4="broad-genomics-delivery/Cancer_Cell_Line_Factory_CCLF_RNAseq"
#workspace5="nci-mimoun-bi-org/CCLF_RNA_2_0"

#workspace3="broad-genomics-delivery/CCLE_DepMap_RNAseq"
#workspace1="broad-genomics-delivery/Getz_IBM_CellLines_RNASeqData"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_RNAseq"
workspace7="terra-broad-cancer-prod/Getz_IBM_CellLines_RNASeqData"

# and their correesponding sample source
source1="ibm"
source2="ccle"
source3="ccle"
source4="cclf"
source5="cclf"
source6="ccle"
source7="ibm"

release = samplesetname

# our working workspace (reference)
refworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"

# info/metadata google spreadsheets (info about cell lines)
refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"
privacy_release_url = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
depmap_pv = "https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE"
depmap_taiga = "arxspan-cell-line-export-f808"

# genomic annotations (v35)
gencode = 'ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_35/gencode.v35.annotation.gff3.gz'

RNAmethods = ['']


replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}
extract_to_change = {'from_arxspan_id': 'participant'}

#version 102
ensemblserver = "http://nov2020.archive.ensembl.org/biomart" 

datatype = 'rna'
match = ['ACH-','CDS-']

# Generate sample set from new samples

we retrieve all the samples we can find from the GP workspaces

__CCLE specific__

In [101]:
tracker.(ccle_refsamples.loc[['CDS-ABH0uZ']],

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_line_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-ABH0uZ,ACH-001767,1,NaN,NaN,rna,311757376.0,NaN,CCLFPEDS0018T,PT-pmZkEqFb,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [100]:
refwm.update_sample_set('re_rsem_21Q2', ['CDS-fk564T',
'CDS-kU30H5',
'CDS-G0F5f5',
'CDS-ABH0uZ'])

APIException: Sample set import failed.: (400) : {
  "causes": [{
    "causes": [],
    "message": "sample CDS-ABH0uZ not found",
    "source": "rawls",
    "stackTrace": []
  }],
  "message": "Could not resolve some entity references",
  "source": "rawls",
  "stackTrace": [],
  "statusCode": 400,
  "timestamp": 1616169136473
}

In [ ]:
samples = loading.loadRNA(samplesetname,workspaces=["terra-broad-cancer-prod/CCLE_DepMap_WES", "terra-broad-cancer-prod/Getz_IBM_CellLines_Exomes"],sources=["ccle", "ibm"],maxage='2020-09-10', baits='polyA',stype="rna")

In [ ]:
loading.update(samples, samplesetname, stype="rna", bucket="", refworkspace,
          name_col="index", values=['legacy_bam_filepath', 'legacy_bai_filepath'],
          filetypes=['bam', 'bai'],
          my_id='~/.client_secret.json',
          mystorage_id="~/.storage.json",
          creds='../.credentials.json',
          sampletrackername='ccle sample tracker', refsheet_url="https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY")

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 6 functions To generate the expression/fusion dataset:

We use the GTEx pipeline ([https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md](https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md)).

To generate the expression dataset, run the following tasks on all samples that you need, in this order:



*   samtofastq_v1-0_BETA_cfg 

    (broadinstitute_gtex/samtofastq_v1-0_BETA Snapshot ID: 5)

*   star_v1-0_BETA_cfg

(broadinstitute_gtex/star_v1-0_BETA Snapshot ID: 7)



*   rsem_v1-0_BETA_cfg 

    (broadinstitute_gtex/rsem_v1-0_BETA Snapshot ID: 4)

*   rsem_aggregate_results_v1-0_BETA_cfg (broadinstitute_gtex/rsem_aggregate_results_v1-0_BETA Snapshot ID: 3)

The outputs to be downloaded will be saved under the sample set that you ran. The outputs we use for the release are:



*   rsem_genes_expected_count
*   rsem_genes_tpm
*   rsem_transcripts_tpm

****Make sure that you delete the intermediate files. These files are quite large so cost a lot to store. To delete, you can either write a task that deletes them or use gsutil rm*****


##### Fusions {#fusions}

We use STAR-Fusion [https://github.com/STAR-Fusion/STAR-Fusion/wiki](https://github.com/STAR-Fusion/STAR-Fusion/wiki). The fusions are generated by running the following tasks



*   hg38_STAR_fusion (gkugener/STAR_fusion Snapshot ID: 14)
*   Aggregate_Fusion_Calls (gkugener/Aggregate_files_set Snapshot ID: 2)

The outputs to be downloaded will be saved under the sample set you ran. The outputs we use for the release are: 



*   fusions_star

This task uses the same samtofastq_v1-0_BETA_cfg task as in the expression pipeline, although in the current implementation, this task will be run twice. It might be worth combing the expression/fusion calling into a single workflow. This task also contains a flag that lets you specify if you want to delete the intermediates (fastqs). 

There are several other tasks in this workspace. In brief:



*   Tasks prefixed with **EXPENSIVE** or **CHEAP** are identical to their non-prefixed version, except that they specify different memory, disk space, etc. parameters. These versions can be used when samples fail the normal version of the task due to memory errors.
*   The following tasks are part of the GTEx pipeline but we do not use them (we use RSEM exclusively): markduplicates_v1-0_BETA_cfg (broadinstitute_gtex/markduplicates_v1-0_BETA Snapshot ID: 2), rnaseqc2_v1-0_BETA_cfg (broadinstitute_gtex/rnaseqc2_v1-0_BETA Snapshot ID: 2)
*   **ExonUsage_hg38_fixed** (gkugener/ExonUsage_fixed Snapshot ID: 1): this task calculates exon usage ratios. The non-fixed version contains a bug in the script that is not able to handle chromosome values prefixed with ‘chr’. The ‘fixed’ version resolves this issue.
*   **AggregateExonUsageRObj_hg38** (ccle_mg/AggregateExonUsageRObj Snapshot ID: 2): combines the exon usage ratios into a matrices that are saved in an R object.

### cleaning workspaces

In [ ]:
torm = terra.listHeavyFiles(refworkspace)
h.parrun(['gsutil rm '+i for i in torm], cores=8)
terra.removeFromFailedWorkflows(refworkspace, dryrun=False, everythingFor)

## On Terra

In [ ]:
# TODO: update with latest workspace parameters from our repo

In [92]:
submission_id = refwm.create_submission("RNA_pipeline", samplesetname,'sample_set',expression='this.samples')
await terra.waitForSubmission(refworkspace, submission_id)

1.0 of jobs Succeeded in submission 0.sion 0. 106 mn elapsed..


[]

In [93]:
submission_id = refwm.create_submission("RNA_aggregate", 'all')
await terra.waitForSubmission(refworkspace, submission_id)

Successfully created submission 4e5d24b2-5b48-453e-9d49-4617028b31f3.
1.0 of jobs Succeeded in submission 0.sion 0. 145 mn elapsed..


[]

## On Local

### Save the workflow configurations used

In [ ]:
terra.saveWorkspace(refworkspace,'data/'+samplesetname+'/RNAconfig/')

### Load QC files

In [ ]:
starlogs = getQC(workspace=refworkspace ,only=samples.index.tolist(), qcname="star_logs",match=".Log.final.out")
rnaqc = getQC(workspace=refworkspace ,only=samples.index.tolist(), qcname="rnaseqc2_metrics")

In [ ]:
for k,val in rnaqc.items():
    if type(val[0]) is not str:
        print("QC was not done for: "+k)

In [ ]:
qcs = pd.DataFrame()
for k,val in rnaqc.items():
    if val[0] is not np.nan:
        qcs = pd.concat([qcs, pd.read_csv(val[0],sep='\t',index_col=0)],axis=1)
qcs = qcs[~((qcs.mean(1)==1.0) | (qcs.mean(1)==0.0))]

In [ ]:
lowqual = rna.filterRNAfromQC(qcs, thresholds={'minmapping': 0.85,
                                          'minendmapping': 0.75,
                                          'minefficiency': 0.75,
                                          'maxendmismatch': 0.02,
                                          'maxmismatch': 0.02,
                                          'minhighqual': 0.8,
                                          'minexon': 0.7,
                                          "maxambiguous": 0.05,
                                          "maxsplits": 0.1,
                                          "maxalt": 0.2,
                                          "maxchim": 0.05,
                                          "minreads": 20000000,
                                          "minlength": 80,
                                          "maxgenes": 35000,
                                          "mingenes": 12000,
                                          }, folder='data/rna_qc_plots/lowqual_'+samplesetname+"/", plot=True, qant1=0.1, qant3=0.9)

In [ ]:
failed = rna.filterRNAfromQC(qcs, thresholds={'minmapping': 0.7,
                                          'minendmapping': 0.66,
                                          'minefficiency': 0.6,
                                          'maxendmismatch': 0.02,
                                          'maxmismatch': 0.02,
                                          'minhighqual': 0.7,
                                          'minexon': 0.66,
                                          "maxambiguous": 0.1,
                                          "maxsplits": 0.1,
                                          "maxalt": 0.5,
                                          "maxchim": 0.2,
                                          "minreads": 20000000,
                                          "minlength": 80,
                                          "maxgenes": 35000,
                                          "mingenes": 10000,
                                          }, folder='data/rna_qc_plots/'+samplesetname+"/", plot=True, qant1=0.07, qant3=0.93)

In [ ]:
failed = failed.index.tolist()

In [ ]:
failed.extend(['CDS-12DTEw', 'CDS-9hv1zM', 'CDS-A6GSeQ', 'CDS-aWlMRt', 'CDS-B1ywOH', 'CDS-BixxtG', 'CDS-DRM3l2', 'CDS-jOlYT4', 'CDS-KMhiT9', 'CDS-M6mnMA', 'CDS-pYwECX', 'CDS-v6E624', 'CDS-vxTqNJ', 'CDS-YxtmkI'])
print('you want to copy that up top, to save it for next time',failed)

In [ ]:
failed

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
for k,v in starlogs.items():
    if k =='nan':
        continue
    if ccle_refsamples.loc[k,'bam_qc']!=v[0]:
        ccle_refsamples.loc[k,'bam_qc']=v[0]

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [ ]:
newfail = []
# finding other replicates to solve failed ones
for val in failed:
    a = ccle_refsamples.loc[val].arxspan_id
    res = ccle_refsamples[(ccle_refsamples.datatype == 'rna') &(ccle_refsamples.arxspan_id == a)]
    if len(res)>1:
        for k in res.index:
            if k not in failed:
                renaming[k] = renaming.pop(val)
    else:
        newfail.append(renaming.pop(val))
failed = newfail
newfail

### Remove some datafile to save money

In [ ]:
res = refwm.get_samples()

In [ ]:
colstoremove = ['fastq1', 'fastq2','recalibrated_bam','recalibrated_bam_index']
for val in colstoremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
samplesinset = samples.index.tolist()
#samplesinset= [i['entityName'] for i in refwm.get_entities('sample_set').loc[samplesetname].samples]

In [ ]:
## copy star bam file to our cclebams/rnasq_hg38/ bucket
renamed, _ = terra.changeGSlocation(workspacefrom=refworkspace, newgs="gs://cclebams/rnasq_hg38/", onlysamples=samplesinset, onlycol=["star_bam_file",'star_bam_index'], entity="sample", keeppath=False,dry_run=False)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
renamed

In [ ]:
ccle_refsamples.loc[samplesinset,['legacy_bam_filepath','legacy_bai_filepath','legacy_size','legacy_crc32c_hash']] = ccle_refsamples.loc[samplesinset][['internal_bam_filepath','internal_bai_filepath','size','crc32c_hash']].values

ccle_refsamples.loc[samplesinset,'internal_bam_filepath'] = renamed['star_bam_file'].values

ccle_refsamples.loc[samplesinset,'internal_bai_filepath'] = renamed['star_bam_index'].values

ccle_refsamples.loc[samplesinset,'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(renamed['star_bam_file'].tolist(),'-l')]
ccle_refsamples.loc[samplesinset,'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(renamed['star_bam_file'].tolist(),'-L')]

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

### Expression post processing

#### loading the files

In [ ]:
res = refwm.get_sample_sets().loc['all']
rsem_genes_expected_count = res['rsem_genes_expected_count']
rsem_genes_tpm = res['rsem_genes_tpm']
rsem_transcripts_tpm = res['rsem_transcripts_tpm']
rsem_transcripts_expected_count = res['rsem_transcripts_expected_count']
! gsutil cp $rsem_genes_expected_count "temp/rsem_genes_expected_count" & gsutil cp $rsem_genes_tpm "temp/rsem_genes_tpm" & gsutil cp $rsem_transcripts_tpm "temp/rsem_transcripts_tpm" & gsutil cp $rsem_transcripts_expected_count "temp/rsem_transcripts_expected_count"

In [ ]:
files = {}
renaming = {}
for val in ["temp/rsem_transcripts_tpm","temp/rsem_genes_tpm", "temp/rsem_genes_expected_count","temp/rsem_transcripts_expected_count"]:
    file = pd.read_csv(val, compression='gzip', header=0, sep='\t', quotechar='"', error_bad_lines=False)
    print(file.columns[:10])
    if len(renaming) == 0:
        # removing failed version
        renaming  = removeOlderVersions(names=file.columns[2:], refsamples=refwm.get_samples(), arxspan_id="arxspan_id", version="version")
        renaming.update({'transcript_id(s)':'transcript'})
        # if we have a replaceable failed version in our dataset
        for k, v in renaming.items():
            if k in rename:
                renaming[rename[k]] = renaming.pop(k)
    if val=="temp/rsem_genes_expected_count":
        replevel = file.drop(columns = 'transcript_id(s)').set_index('gene_id', drop=True)
        replevel = replevel[(replevel.sum(1) != 0) & (replevel.var(1) != 0)]
    # we remove the failed samples where we did not found anything else to replace them with
    files[val.split('/')[-1]] = file[file.columns[:2].tolist()+[i for i in file.columns[2:] if i in set(renaming.keys())]].rename(columns=renaming)

#### generating gene names

In [ ]:
server = BiomartServer(ensemblserver)
ensmbl = server.datasets['hsapiens_gene_ensembl']
ensembltohgnc = pd.read_csv(io.StringIO(ensmbl.search({
  'attributes': ['ensembl_gene_id','clone_based_ensembl_gene','hgnc_symbol','gene_biotype','entrezgene_id']
}, header=1).content.decode()), sep='\t')

ensembltohgnc.columns = ['ensembl_gene_id','clone_based_ensembl_gene','hgnc_symbol','gene_biotype','entrezgene_id']
ensembltohgnc = ensembltohgnc[~(ensembltohgnc['clone_based_ensembl_gene'].isna() & ensembltohgnc['hgnc_symbol'].isna())]
ensembltohgnc.loc[ensembltohgnc[ensembltohgnc.hgnc_symbol.isna()].index,"hgnc_symbol"] = ensembltohgnc[ensembltohgnc.hgnc_symbol.isna()]['clone_based_ensembl_gene']

gene_rename =  {i.ensembl_gene_id: i.hgnc_symbol+' ('+i.ensembl_gene_id+')' for k,i in ensembltohgnc.iterrows()}

In [ ]:
protcod_rename = {i.ensembl_gene_id: i.hgnc_symbol+' ('+str(int(i.entrezgene_id))+')' for _,i in ensembltohgnc[(~ensembltohgnc.entrezgene_id.isna()) & (ensembltohgnc.gene_biotype=='protein_coding')].iterrows()}

#### renaming the files

In [ ]:
# gene level
for val in ['rsem_genes_expected_count','rsem_genes_tpm']:
    file = files[val].drop(columns='transcript').set_index('gene_id')
    file = file[(file.sum(1) != 0) & (file.var(1) != 0)]
    r = [i.split('.')[0] for i in file.index]
    dup = h.dups(r)
    if len(dup)>0:
        print(dup)
        raise ValueError('duplicate genes')
    file.index = r
    files[val.replace('genes','proteincoding_genes')] = file[file.index.isin(set(ensembltohgnc[ensembltohgnc.gene_biotype == 'protein_coding'].ensembl_gene_id))]
    files[val] = file.rename(index=gene_rename).T
    files[val.replace('genes','proteincoding_genes')] = files[val.replace('genes','proteincoding_genes')].rename(index=protcod_rename).T

In [ ]:
# transcript level
rename_transcript = {}
missing = []
for val in ['rsem_transcripts_expected_count','rsem_transcripts_tpm']:
    file = files[val]
    file = file[(file[file.columns[2:]].sum(1) != 0) & (file[file.columns[2:]].var(1) != 0)]
    r = [i.split('.')[0] for i in file.transcript_id]
    dup = h.dups(r)
    if len(dup)>0:
        print(dup)
        raise ValueError('duplicate genes')    
    file.transcript_id = r
    if len(rename_transcript)==0:
        for _,v in file.iterrows():
            if v.gene_id.split('.')[0] in gene_rename:
                rename_transcript[v.transcript_id] = gene_rename[v.gene_id.split('.')[0]].split(' (')[0] + ' (' + v.transcript_id + ')'
            else:
                missing.append(v.gene_id.split('.')[0])
        print('missing: '+str(len(missing))+' genes')
    files[val] = file.set_index('transcript_id').drop(columns = 'gene_id').rename(index = rename_transcript).T

In [ ]:
# removing genes that did not match.. pretty unfortunate 
files['rsem_proteincoding_genes_expected_count'] = files['rsem_proteincoding_genes_expected_count'][[i for i in files['rsem_proteincoding_genes_expected_count'].columns if ' (' in i]]
files['rsem_proteincoding_genes_tpm'] = files['rsem_proteincoding_genes_tpm'][[i for i in files['rsem_proteincoding_genes_tpm'].columns if ' (' in i]]

In [ ]:
# check: do we have any duplicates?
h.dups(files['rsem_proteincoding_genes_expected_count'].columns.tolist())

In [ ]:
# if we do, managing duplicates
for val in ['rsem_proteincoding_genes_expected_count', 'rsem_proteincoding_genes_tpm']:
    for dup in h.dups(files[val].columns):
        a = files[val][dup].sum()
        files[val].drop(columns=dup)
        files[val][dup] = a

#### saving samples used for the release

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
ccle_refsamples.loc[set(renaming.keys())-set(['transcript_id(s)']),release]=1
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype=='rna'].index,'low_quality']=0
ccle_refsamples.loc[lowqual[lowqual.sum(1)>3].index.tolist(),'low_quality']=1
ccle_refsamples.to_csv('temp/updated_ref_samples.csv')

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

### validation

In [ ]:
files.keys()

In [ ]:
prevcounts = tc.get(name='depmap-a0ab', file='CCLE_RNAseq_reads')
overlap = set(prevcounts.columns) & set(files['rsem_genes_expected_count'].columns)
len(overlap)

In [ ]:
# have we lost any files compared to last release?
lost = set(prevcounts.index) - set(files['rsem_proteincoding_genes_tpm'].index)
%store failed
lost

In [ ]:
# do we have samples that are missanotated compared to previous releases (replicate level)
notindataset, missannotated, unmatched = findMissAnnotatedReplicates(replevel, prevcounts, renaming)

In [ ]:
for k,v in unmatched.items():
    if ccle_refsamples.loc[k].arxspan_id!=v:
        print(k,v)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
normals = ccle_refsamples[ccle_refsamples['primary_disease']=='normal'].index.tolist()
normals

In [ ]:
#for k,val in files.items():
    #files[k] = val.drop(index=normals)

In [ ]:
# do we have samples that are missanotated compared to previous releases (sample level)
unmatched = rna.getDifferencesFromCorrelations(files['rsem_genes_expected_count'] ,prevcounts, minsimi=0.95)
unmatched

In [ ]:
# Is it because of  duplicate version?
rnasamples = ccle_refsamples[ccle_refsamples.datatype=='rna']
for i,val in unmatched:
    print(len(rnasamples[rnasamples.arxspan_id==i]))

### ssGSEA

In [ ]:
counts_genes = files['rsem_genes_expected_count'].copy()

In [ ]:
counts_genes

In [ ]:
[i for i in counts_genes.columns if 'MYC' in i]

In [ ]:
counts_genes.columns = [i.split(' (')[0] for i in counts_genes.columns]

In [ ]:
for i in h.dups(counts_genes.columns):
    val = counts_genes[i].sum(1)
    counts_genes=counts_genes.drop(columns=i)
    counts_genes[i]=val

In [ ]:
len(set([val for val in counts_genes.columns if '.' not in val]))

In [ ]:
# MAYBE NOT NEEDED
#### merging splicing variants into the same gene
#counts_genes_merged, _, _= h.mergeSplicingVariants(counts_genes.T, defined='.')
counts_genes

#### with R ssGSEA on the console

In [ ]:
enrichments= rna.gsva(counts_genes.T, pathtoJKBio="../JKBio/", geneset_file = "data/genesets/msigdb.v7.2.symbols.gmt", method='ssgsea').T
enrichments.index = [i.replace('.','-') for i in enrichments.index]

### Some predictions

In [ ]:
metadata = tc.get(name='depmap-a0ab', file='sample_info')
metadata

In [ ]:
# finding train and test set
trainame = [val for val in new1&prev if val[:3] == 'ACH']
testname = [val for val in new1-prev if val[:3] == 'ACH']

#looking at the 2000 most variable genes in the two sets
genetolookfor = 2000
gene_var = counts_genes[trainame].var(1).values
print(len(gene_var))
sorting = np.argsort(gene_var)[-genetolookfor:]

In [ ]:
unregistered = set(testname) - set(metadata["DepMap_ID"].values.tolist())
unregistered

In [ ]:
len(counts_genes['ACH-001767']) - np.count_nonzero(counts_genes['ACH-001767'].values)

In [ ]:
# creating and reodering train and test sets
traindata = counts_genes[set(trainame)-unregistered].values[sorting].T
trainlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(trainame)-unregistered].columns.tolist() if val not in unregistered]

testdata = counts_genes[set(testname)-unregistered].values[sorting].T
testlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(testname)-unregistered].columns.tolist() if val not in unregistered]
traindata.shape

In [ ]:
# learn KNN classifier to the metadata diseases
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(traindata, trainlabels) 
predicted = neigh.predict(testdata)
predicted[1]

In [ ]:
labels = trainlabels + testlabels
colors=[0]*len(trainlabels)
colors.extend([1,2,2,2,2,1,2,2,2,1,2])
data = np.vstack([traindata,testdata])

In [ ]:
# plot them with TSNE, highlight the points that failed and show colors for diseases
dimred = TSNE(2,10).fit_transform(data)

In [ ]:
scatter(dimred, labels=labels,colors=colors, radi=1.9)

### Save files for taiga

In [ ]:
!ls temp/gene_sets_*

In [ ]:
#h.compareDfs(enrichments, tc.get(name='depmap-a0ab', file='CCLE_fusions_unfiltered'))

In [ ]:
len(set(ccle_refsamples[ccle_refsamples.datatype=='rna'].arxspan_id))

In [ ]:
enrichments.to_csv('temp/gene_sets_'+release+'_all.csv')

In [ ]:
#CCLE_expression, CCLE_expression_full, , 
#h.compareDfs(files["rsem_transcripts_tpm"], tc.get(name='depmap-a0ab', file='CCLE_RNAseq_transcripts'))
#h.compareDfs(files["rsem_transcripts_expected_count"], tc.get(name='depmap-a0ab', file='CCLE_expression_transcripts_expected_count'))
h.compareDfs(files["rsem_genes_expected_count"], tc.get(name='depmap-a0ab', file='CCLE_RNAseq_reads'))
h.compareDfs(files["rsem_genes_tpm"], tc.get(name='depmap-a0ab', file='CCLE_expression_full'))
h.compareDfs(files["rsem_proteincoding_genes_tpm"], tc.get(name='depmap-a0ab', file='CCLE_expression'))
h.compareDfs(files["rsem_proteincoding_genes_expected_count"], tc.get(name='depmap-a0ab', file = 'CCLE_expression_proteincoding_genes_expected_count'))

In [ ]:
for k,val in files.items():
    #h.compareDfs(val, tc.get(name='depmap-a0ab', file=k))
    val.to_csv('temp/'+k.replace('rsem','expression_'+release)+'.csv')

In [ ]:
tc.update_dataset(dataset_permaname="expression-d035",
                 upload_file_path_dict={
    'temp/expression_'+release+'_transcripts_tpm.csv': 'NumericMatrixCSV',
    'temp/expression_'+release+'_genes_tpm.csv': 'NumericMatrixCSV',
    'temp/expression_'+release+'_genes_expected_count.csv': 'NumericMatrixCSV',
    'temp/expression_'+release+'_proteincoding_genes_expected_count.csv': 'NumericMatrixCSV',
    'temp/expression_'+release+'_proteincoding_genes_tpm.csv': 'NumericMatrixCSV',
    'temp/expression_'+release+'_transcripts_expected_count.csv': 'NumericMatrixCSV',
    'temp/gene_sets_'+release+'_all.csv': 'NumericMatrixCSV'
                },
                  dataset_description=
"""
# RNAseq

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal.

/!\ This is the most up to date version of the CCLE RNA data.

## Annotations:

transcriptions (Transcripts rpkm):

genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
""")

## Fusion post processing

In [ ]:
terra.saveConfigs(refworkspace,'data/'+samplesetname+'/RNAconfig')

In [ ]:
terra.waitForSubmission(refworkspace, submission_id2)

In [ ]:
aggregated = refwm.get_sample_sets().loc['all']['fusions_star']

In [ ]:
! gsutil cp $aggregated "temp/expression.fusion.tsv"

In [ ]:
fusions = pd.read_csv('temp/expression.fusion.tsv', names=['DepMap_ID', 'FusionName', 'JunctionReadCount', 'SpanningFragCount', 'SpliceType', 'LeftGene', 'LeftBreakpoint', 'RightGene', 'RightBreakpoint', 'LargeAnchorSupport', 'FFPM', 'LeftBreakDinuc', 'LeftBreakEntropy', 'RightBreakDinuc', 'RightBreakEntropy','annots'], skiprows=1, sep='\t')
fusions["DepMap_ID"] = [i.split('.')[0] for i in fusions['DepMap_ID']]

In [ ]:
print(len(fusions))
renaming = removeOlderVersions(names=set(fusions['DepMap_ID']), refsamples=refwm.get_samples(), arxspan_id="arxspan_id", version="version")
print(len(fusions))
print(fusions['DepMap_ID'][:10])

In [ ]:
fusions['CCLE_count'] = [i.LeftBreakpoint+'_'+i.RightBreakpoint for k, i in fusions.iterrows()]
counts = Counter(list(fusions['CCLE_count']))
fusions['CCLE_count'] = [counts[val] for val in fusions['CCLE_count']]

In [ ]:
sns.kdeplot(fusions['CCLE_count'] )

In [ ]:
fusions

In [ ]:
fusions.RightGene = renameFusionGene(fusions.RightGene)
fusions.LeftGene = renameFusionGene(fusions.LeftGene)

In [ ]:
fusions[['LeftGene', 'RightGene']] = fusions[['LeftGene', 'RightGene']].applymap(lambda x: re.sub(r'([^\^]+)\^(.*)$', r'\1 (\2)', x))

In [ ]:
fusions.to_csv('temp/fusions_withReplicates_'+release+'.csv',index=False)

In [ ]:
fusions = pd.read_csv('temp/fusions_withReplicates_'+release+'.csv')

### Generate filtered fusion table

We want to apply filters to the fusion table to reduce the number of artifacts in the dataset. Specifically, we filter the following:

* Remove fusions involving mitochondrial chromosomes, or HLA genes, or immunoglobulin genes
* Remove red herring fusions (from STAR-Fusion annotations column)
* Remove recurrent in CCLE (>= 25 samples)
* Remove fusion with (SpliceType=" INCL_NON_REF_SPLICE" and LargeAnchorSupport="No" and FFPM < 0.1)
* Remove fusions with FFPM < 0.05 (STAR-Fusion suggests using 0.1, but looking at the translocation data, this looks like it might be too aggressive)


In [ ]:
fusions = fusions[fusions['DepMap_ID'].isin(renaming.keys())].replace({'DepMap_ID':renaming}).reset_index(drop=True)

In [ ]:
fusions['CCLE_count'] = [i.LeftBreakpoint+'_'+i.RightBreakpoint for k, i in fusions.iterrows()]
counts = Counter(list(fusions['CCLE_count']))
fusions['CCLE_count'] = [counts[val] for val in fusions['CCLE_count']]

In [ ]:
filtered = filterFusions(fusions, maxfreq=0.1)

In [ ]:
len(set(fusions.DepMap_ID))

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
normals = ccle_refsamples[ccle_refsamples['primary_disease']=='normal'].index.tolist()
normals

In [ ]:
#fusions = fusions[~fusions.DepMap_ID.isin(normals)]
#filtered = filtered[~filtered.DepMap_ID.isin(normals)]

In [ ]:
a = tc.get(name='depmap-a0ab', file='CCLE_fusions_unfiltered')
print('new')
print(set(fusions.DepMap_ID) - set(a.DepMap_ID))
print('removed')
print(set(a.DepMap_ID) - set(fusions.DepMap_ID))
print("in fusion, not in rna")
print(set(fusions.DepMap_ID) - set(files['rsem_proteincoding_genes_tpm'].index))
print('in depmap, not in fusions')
print(set(files['rsem_proteincoding_genes_tpm'].index) - set(fusions.DepMap_ID))

In [ ]:
fusions.to_csv('temp/fusions_'+release+'.csv',index=False)
filtered.to_csv('temp/filtered_fusions_'+release+'.csv',index=False)

In [ ]:
fusions = pd.read_csv('temp/fusions_'+release+'.csv')
filtered = pd.read_csv('temp/filtered_fusions_'+release+'.csv')

### Uploading to Taiga

In [ ]:
tc.update_dataset(dataset_permaname="fusions-95c9",
                     upload_file_path_dict={
                'temp/fusions_'+release+'.csv': 'TableCSV',
                'temp/filtered_fusions_'+release+'.csv': 'TableCSV',
                'temp/fusions_withReplicates_'+release+'.csv': "TableCSV"},
                 dataset_description="""
# Fusions

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal.

/!\ This is the most up to date version of the CCLE CN data.

## Annotations

Description: Gene fusions derived from RNAseq data.

Rows: cell lines, IDs contained in the column DepMap_ID

Unfiltered data contains all output fusions, while the filtered data uses the filters suggested by the star fusion docs. These filters are:
- FFPM > 0.1 -  a cutoff of 0.1 means&nbsp;at least 1 fusion-supporting RNAseq fragment per 10M total reads
- Remove known false positives, such as GTEx recurrent fusions and certain paralogs
- Genes that are next to each other
- Fusions with mitochondrial breakpoints
- Removing fusion involving mitochondrial chromosomes or HLA genes
- Removed common false positive fusions (red herring annotations as described in the STAR-Fusion docs)
- Recurrent fusions observed in CCLE across cell lines (in more than 10% of our samples)
- Removed fusions where SpliceType='INCL_NON_REF_SPLICE' and LargeAnchorSupport='NO_LDAS' and FFPM < 0.1
- FFPM < 0.05
""")